# Creating an agent with MCP

The [Model Context Protocol](https://www.anthropic.com/news/model-context-protocol) (MCP) introduced by Anthropic has proven to be a popular method for providing an AI agent with access to a variety of tools. [This Huggingface blog post ](https://huggingface.co/blog/Kseniase/mcp) has a nice explanation of MCP.  In this tutorial, we'll build an agent that is able to leverage MCP server provided tools.

Note: because this tutorial relies upon advanced stdio/stderr communication using the MCP Server, it cannot be run on Google Colab.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks, this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [ ]:
%pip install 'any-agent[all]'

import nest_asyncio

nest_asyncio.apply()

## Configure the Agent

Now it's time to configure the agent! At this stage you have a few choices:

### Pick the framework

We support a variety of underlying agent frameworks (OpenAI, Smolagents, Langchain, etc), which all have their own particular agentic AI implementations. For this tutorial's simple use case, any of the frameworks should work just fine, but any-agent makes it easy to try out a different framework later, if we so choose. For this example, we will use the "openai" agent framework.

### Pick an LLM

Regardless of which agent framework you choose, each framework supports LiteLLM, which is a proxy that allows us to use whichever LLM inside the framework, hosted on by any provider. For example, we could use a local model via llama.cpp or llamafile, a google hosted gemini model, or a AWS bedrock hosted Llama model. For this example, let's use OpenAI's gpt-4.1-nano.

### Pick which tools to use

 In this example, we'll add a few MCP servers that we host locally, which means we'll use a Stdio MCP server. If an MCP Server is already running and hosted elsewhere, you can use an SSE connection to access it. You can browse some of the officially supported MCP servers [here](https://github.com/modelcontextprotocol/servers/tree/main?tab=readme-ov-file).

 Lets give use two MCP servers: 
 
 * [Time](https://github.com/modelcontextprotocol/servers/tree/main/src/time): so the agent can know what time/day it is.
 * [Airbnb](https://github.com/openbnb-org/mcp-server-airbnb): so the agent can browse airbnb listings

 I will also add a custom send_message tool, that way it can ask us additional questions before getting its final answer!
  

In [8]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    print("OPENAI_API_KEY not found in environment!")
    api_key = getpass("Please enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("OPENAI_API_KEY set for this session!")
else:
    print("OPENAI_API_KEY found in environment.")

OPENAI_API_KEY found in environment.


In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.config import MCPStdio

time_tool = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=[
        "get_current_time",
    ],
)

airbnb_tool = MCPStdio(
    command="npx", args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"]
)


def send_message(message: str) -> str:
    """Display a message to the user and wait for their response.

    Args:
        message: str
            The message to be displayed to the user.

    Returns:
        str: The response from the user.

    """
    return input(message + " ")


agent = AnyAgent.create(
    "openai",  # See all options in https://mozilla-ai.github.io/any-agent/
    AgentConfig(model_id="gpt-4o", tools=[airbnb_tool, time_tool, send_message]),
)

## Run the Agent

Now we've configured our agent, so it's time to run it! Let's give it a simple task: find 5 trending new TV shows that were released recently.


In [11]:
prompt = """
I am looking to book an airbnb next weekend near Ohiopyle, PA. Can you help me plan this? Figure out the time, then ask me some questions and lets figure this out together.
"""

agent_trace = agent.run(prompt)

────────────────────────────────────────────────────── TOOL ───────────────────────────────────────────────────────

tool_name: get_current_time

input: {'timezone': 'America/New_York'}

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ {"type":"text","text":"{\n  "timezone": "America/New_York",\n  "datetime": "2025-05-21T15:31:05-04:00",\n       │
│ "is_dst": true\n}","annotations":null}                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ {"type":"text","text":"{\n  "searchUrl":                                                                        │
│ "https://www.airbnb.com/s/Ohiopyle%2C%20PA/homes?checkin=2025-05-24&checkout=2025-05-25&adults=2&children=1&inf │
│ ants=1&pets=1",\n  "searchResults": [\n    {\n      "id": "1049325800817947598",\n      "url":                  │
│ "https://www.airbnb.com/rooms/1049325800817947598",\n      "demandStayListing": {\n        "id":                │
│ "RGVtYW5kU3RheUxpc3Rpbmc6MTA0OTMyNTgwMDgxNzk0NzU5OA==",\n        "description": {\n          "name": {\n        │
│ "localizedStringWithTranslationPreference": "River's End | 1BR,1BA, Cabinesque Motel in Ohiopyl"\n          }\n │
│ },\n        "location": {\n          "coordinate": {\n            "latitude": 39.86954,\n                       │
│ "longitude": -79.49283\n          }\n        }\n      },\n      "badges": "",\n      "structuredContent": {\n   │
│ "mapCategoryInfo": "",\n        "mapSecondaryLine": "",\n        "primaryLine": "2 beds, 1 bedroom",\n          │
│ "secondaryLine": ""\n      },\n      "structuredDisplayPrice": {\n        "primaryLine": {\n                    │
│ "accessibilityLabel": "$242 for 1 night"\n        },\n        "explanationData": {\n          "title": "Price   │
│ details",\n          "priceDetails": "$242 x 1 night: $242"\n        }\n      }\n    },\n    {\n      "id":     │
│ "781927184983955748",\n      "url": "https://www.airbnb.com/rooms/781927184983955748",\n                        │
│ "demandStayListing": {\n        "id": "RGVtYW5kU3RheUxpc3Rpbmc6NzgxOTI3MTg0OTgzOTU1NzQ4",\n                     │
│ "description": {\n          "name": {\n            "localizedStringWithTranslationPreference": "Evergreen Lodge │
│ Golfing-Hiking-Swimming and more!"\n          }\n        },\n        "location": {\n          "coordinate": {\n │
│ "latitude": 40.06116,\n            "longitude": -79.26063\n          }\n        }\n      },\n      "badges":    │
│ "Guest favorite",\n      "structuredContent": {\n        "mapCategoryInfo": "",\n        "mapSecondaryLine":    │
│ "",\n        "primaryLine": "4 beds, 2 bedrooms",\n        "secondaryLine": ""\n      },\n                      │
│ "avgRatingA11yLabel": "4.92 out of 5 average rating,  114 reviews",\n      "structuredDisplayPrice": {\n        │
│ "primaryLine": {\n          "accessibilityLabel": "$331 for 1 night, originally $430"\n        },\n             │
│ "explanationData": {\n          "title": "Price details",\n          "priceDetails": "$331 x 1 night: $331, "\n │
│ }\n      }\n    },\n    {\n      "id": "24848671",\n      "url": "https://www.airbnb.com/rooms/24848671",\n     │
│ "demandStayListing": {\n        "id": "RGVtYW5kU3RheUxpc3Rpbmc6MjQ4NDg2NzE=",\n        "description": {\n       │
│ "name": {\n            "localizedStringWithTranslationPreference": "Hoover Suite- GAP Trail Hotel at Morguen    │
│ Toole Co"\n          }\n        },\n        "location": {\n          "coordinate": {\n            "latitude":   │
│ 39.814388,\n            "longitude": -79.025269\n          }\n        }\n      },\n      "badges": "",\n        │
│ "structuredContent": {\n        "mapCategoryInfo": "",\n        "mapSecondaryLine": "",\n        "primaryLine": │
│ "2 beds, 2 bedrooms",\n        "secondaryLine": ""\n      },\n      "avgRatingA11yLabel": "4.79 out of 5        │
│ average rating,  85 reviews",\n      "structuredDisplayPrice": {\n        "primaryLine": {\n                    │
│ "accessibilityLabel": "$204 for 1 night"\n        },\n        "explanationData": {\n          "title": "Price   │
│ details",\n          "priceDetails": "$204 x 1 night: $204"\n        }\n      }\n    },\n    {\n      "id":     │
│ "51538566",\n      "url": "https://www.airbnb.com/rooms/51538566",\n      "demandStayListing": {\n        "id": │
│ "RGVtYW5kU3RheUxpc3Rpbmc6NTE1Mzg1NjY=",\n        "desc

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

─────────────────────────────────────────────────────── LLM ───────────────────────────────────────────────────────

╭──────────────────────────────────────────────────── Output ─────────────────────────────────────────────────────╮
│ Here are some options for Airbnb stays near Ohiopyle, PA, for your trip next weekend with 2 adults, 1 child, 1  │
│ infant, and 1 dog. These options are in woodsy areas:                                                           │
│                                                                                                                 │
│  1 ]8;id=503619;https://www.airbnb.com/rooms/1049325800817947598\River's End | 1BR,1BA, Cabinesque Motel in Ohiopyl]8;;\                                                           │
│     • 2 beds, 1 bedroom                                                                                         │
│     • Price: $242 for 1 night                                                                                   │
│     • Location: ]8;id=678595;https://maps.google.com/?q=39.86954,-79.49283\Map]8;;\                                                                                             │
│  2 ]8;id=704232;https://www.airbnb.com/rooms/781927184983955748\Evergreen Lodge Golfing-Hiking-Swimming and more!]8;;\                                                            │
│     • 4 beds, 2 bedrooms                                                                                        │
│     • Price: $331 for 1 night (originally $430)                                                                 │
│     • Guest favorite                                                                                            │
│     • Location: ]8;id=240573;https://maps.google.com/?q=40.06116,-79.26063\Map]8;;\                                                                                             │
│  3 ]8;id=685119;https://www.airbnb.com/rooms/24848671\Hoover Suite- GAP Trail Hotel at Morguen Toole Co]8;;\                                                            │
│     • 2 beds, 2 bedrooms                                                                                        │
│     • Price: $204 for 1 night                                                                                   │
│     • Location: ]8;id=648483;https://maps.google.com/?q=39.814388,-79.025269\Map]8;;\                                                                                             │
│  4 ]8;id=933130;https://www.airbnb.com/rooms/1151299878283759623\Pet Friendly Creekside Cabin Fisherman's Paradise]8;;\                                                            │
│     • 8 beds, 4 bedrooms                                                                                        │
│     • Price: $615 for 1 night (originally $666)                                                                 │
│     • Location: ]8;id=933757;https://maps.google.com/?q=40.05588,-79.22279\Map]8;;\                                                                                             │
│  5 ]8;id=372286;https://www.airbnb.com/rooms/45530398\Cozy Up in this Wooded Laurel Highlands Retreat]8;;\                                                              │
│     • Rating: 4.9 out of 5, 115 reviews                                                                         │
│     • Price: $370 for 1 night                                                                                   │
│     • Location: 1 min walk to park                                                                              │
│     • Location: ]8;id=887667;https://maps.google.com/?q=39.98278,-79.22703\Map]8;;\                                                                                             │
│                                                                                                                 │
│ Take a look at these options, and let me know if any of them suit your needs or if you want more information on │
│ any. You can also visit the ]8;id=2996;https://www.airbnb.com/s/Ohiopyle%2C

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [12]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")
print(f"Usage: {agent_trace.usage}")
print(f"Cost (USD): {agent_trace.cost.total_cost:.6f}")

Here are some options for Airbnb stays near Ohiopyle, PA, for your trip next weekend with 2 adults, 1 child, 1 infant, and 1 dog. These options are in woodsy areas:

1. **[River's End | 1BR,1BA, Cabinesque Motel in Ohiopyl](https://www.airbnb.com/rooms/1049325800817947598)**  
   - 2 beds, 1 bedroom  
   - Price: $242 for 1 night  
   - Location: [Map](https://maps.google.com/?q=39.86954,-79.49283)

2. **[Evergreen Lodge Golfing-Hiking-Swimming and more!](https://www.airbnb.com/rooms/781927184983955748)**  
   - 4 beds, 2 bedrooms  
   - Price: $331 for 1 night (originally $430)  
   - Guest favorite  
   - Location: [Map](https://maps.google.com/?q=40.06116,-79.26063)

3. **[Hoover Suite- GAP Trail Hotel at Morguen Toole Co](https://www.airbnb.com/rooms/24848671)**  
   - 2 beds, 2 bedrooms  
   - Price: $204 for 1 night  
   - Location: [Map](https://maps.google.com/?q=39.814388,-79.025269)

4. **[Pet Friendly Creekside Cabin Fisherman's Paradise](https://www.airbnb.com/rooms/1151299